In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
df=pd.read_csv('../input/quora-question-pairs/train.csv.zip')
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

* In question2 , we have 2 null values
* In question 1, we have 1 null value

In [ ]:
df.groupby('is_duplicate')['id'].count().plot.bar()

percentage of 0's and 1's in is_duplicate label

In [ ]:
dup_0=df[df['is_duplicate']==0].shape[0]
dup_1=df.shape[0]-dup_0

In [ ]:
print("Percentage of 0's or not duplicates in dataset: {0:.2f}".format(dup_0*100/(dup_0+dup_1)))
print("Percentage of 1's or duplicates in dataset: {0:.2f}".format(dup_1*100/(dup_0+dup_1)))

In [ ]:
#number of unique questions in the dataset
from collections import Counter
q_list1=df['qid1'].tolist()
q_list2=df['qid2'].tolist()
q_list1.extend(q_list2)
c=Counter(q_list1)


In [ ]:
print("Number of unique questions in dataset: ",len(c.values()))
print("Maximum number of times a question is repeated :  ",max(c.values()))
print("Number of times questions are repeated in dataset: ",sum(c.values())-len(c.values()))
t=0
for i in c.values():
    if i>1:
        t+=1
print("Number of unique questions that are repeated: ",t)

In [ ]:
#checking whether there are any duplicate pairs
group_df_count=df.groupby(['qid1','qid2']).count().shape[0]
print("Number of duplicate pairs: ",group_df_count-df.shape[0])

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(c.values(),bins=160)
plt.yscale('log',nonposy='clip')
plt.title('log-histogram of questions occurrences')
plt.xlabel('number of occurrences of questions')
plt.ylabel('total number of questions')
plt.show()

### The above graph shows that there are a lot of questions which are not repeated

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.fillna(' ',inplace=True)


In [ ]:
df[df.isnull().any(1)]

No more null values are present

Let us now construct a few features like:
 - ____freq_qid1____ = Frequency of qid1's
 - ____freq_qid2____ = Frequency of qid2's 
 - ____q1len____ = Length of q1
 - ____q2len____ = Length of q2
 - ____q1_n_words____ = Number of words in Question 1
 - ____q2_n_words____ = Number of words in Question 2
 - ____word_Common____ = (Number of common unique words in Question 1 and Question 2)
 - ____word_Total____ =(Total num of words in Question 1 + Total num of words in Question 2)
 - ____word_share____ = (word_common)/(word_Total)
 - ____freq_q1+freq_q2____ = sum total of frequency of qid1 and qid2 
 - ____freq_q1-freq_q2____ = absolute difference of frequency of qid1 and qid2 

In [ ]:
df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count') 
df['freq_qid2'] = df.groupby('qid2')['qid2'].transform('count')
df['q1len'] = df['question1'].str.len() 
df['q2len'] = df['question2'].str.len()
df['q1_n_words'] = df['question1'].apply(lambda row: len(row.split(" ")))
df['q2_n_words'] = df['question2'].apply(lambda row: len(row.split(" ")))
def normalized_word_Common(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)
df['word_Common'] = df.apply(normalized_word_Common, axis=1)
def normalized_word_Total(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * (len(w1) + len(w2))
df['word_Total'] = df.apply(normalized_word_Total, axis=1)
def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))
df['word_share'] = df.apply(normalized_word_share, axis=1)
df['freq_q1+q2'] = df['freq_qid1']+df['freq_qid2']
df['freq_q1-q2'] = abs(df['freq_qid1']-df['freq_qid2'])

In [ ]:
df.head()

Let's check out minimum length of question1 and question2 and  number of questions having that length 

In [ ]:
print("minimum length of question 1: ",df['q1_n_words'].min())
print("minimum length of question 2: ",df['q2_n_words'].min())
t1=df['q1_n_words'].min()
t2=df['q2_n_words'].min()

In [ ]:
print("number of question 1 with minimum length: ",df[df['q1_n_words']==t1].shape[0])
print("number of question 2 with minimum length: ",df[df['q2_n_words']==t2].shape[0])

### Feature: word_share 

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='word_share',data=df)
plt.title('violin plot')
plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate']==1]['word_share'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['word_share'],color='blue',label='0-not duplicate')
plt.title('PDF for word_share')
plt.legend()
plt.show()


### We can see that as the word_share increases, there are more chances they are similar

### Next feature : word_Common

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='word_Common',data=df)
plt.title('Violin plot for word_Common')
plt.subplot(1,2,2)
plt.title('PDF for word_Common')
sns.distplot(df[df['is_duplicate']==1]['word_Common'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['word_Common'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


### We can see that graphs are highly overlapping, word_Common may not be that useful as compared to word_share

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.title('Violin plot for freq_q1+q2')
sns.violinplot(x='is_duplicate',y='freq_q1+q2',data=df)
plt.subplot(1,2,2)
plt.title('PDF for freq_q1+q2')
sns.distplot(df[df['is_duplicate']==1]['freq_q1+q2'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['freq_q1+q2'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.title('Violin plot for freq_q1-q2')
sns.violinplot(x='is_duplicate',y='freq_q1-q2',data=df)
plt.subplot(1,2,2)
plt.title('PDF for freq_q1-q2')
sns.distplot(df[df['is_duplicate']==1]['freq_q1-q2'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['freq_q1-q2'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


### Both sum as well as difference of frequencies graphs are highly overlapping , therefore they may not be useful 

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.title('Violin plot for q1_n_words')
sns.violinplot(x='is_duplicate',y='q1_n_words',data=df)
plt.subplot(1,2,2)
plt.title('PDF for q1_n_words')
sns.distplot(df[df['is_duplicate']==1]['q1_n_words'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['q1_n_words'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='q2_n_words',data=df)
plt.title('Violin plot for q2_n_words')
plt.subplot(1,2,2)
plt.title('PDF for q2_n_words')
sns.distplot(df[df['is_duplicate']==1]['q2_n_words'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['q2_n_words'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.title('Violin plot for q1len')
sns.violinplot(x='is_duplicate',y='q1len',data=df)
plt.subplot(1,2,2)
plt.title('PDF for q1len')
sns.distplot(df[df['is_duplicate']==1]['q1len'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['q1len'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.title('Violin plot for q2len')
sns.violinplot(x='is_duplicate',y='q2len',data=df)
plt.subplot(1,2,2)
plt.title('PDF for q2len')
sns.distplot(df[df['is_duplicate']==1]['q2len'],color='red',label='1-duplicate')
sns.distplot(df[df['is_duplicate']==0]['q2len'],color='blue',label='0-not duplicate')
plt.legend()
plt.show()


All of the graphs except word_share are highly overlapping

## Text Preprocessing

- Preprocessing:
    - Removing html tags   -> (removing angular brackets )
    - Removing Punctuations   -> (comma , full-stop , exclaimation marks etc)
    - Performing stemming  -> (reducing a word to its base form)
    - Removing Stopwords  -> (removing words which don't add meaning)
    - Expanding contractions etc.-> (replace won't with will not , isn't with is not and so on)

In [ ]:
# To get the results in 4 decemal points
SAFE_DIV = 0.0001 

STOP_WORDS = stopwords.words("english")


def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x
    

Definition:
- __Token__: You get a token by splitting sentence a space
- __Stop_Word__ : stop words as per NLTK.
- __Word__ : A token that is not a stop_word


Features:
- __cwc_min__ :  Ratio of common_word_count to min lenghth of word count of Q1 and Q2 <br>cwc_min = common_word_count / (min(len(q1_words), len(q2_words))
<br>
<br>
- __cwc_max__ :  Ratio of common_word_count to max lenghth of word count of Q1 and Q2 <br>cwc_max = common_word_count / (max(len(q1_words), len(q2_words))
<br>
<br>
- __csc_min__ :  Ratio of common_stop_count to min lenghth of stop count of Q1 and Q2 <br> csc_min = common_stop_count / (min(len(q1_stops), len(q2_stops))
<br>
<br>
- __csc_max__ :  Ratio of common_stop_count to max lenghth of stop count of Q1 and Q2<br>csc_max = common_stop_count / (max(len(q1_stops), len(q2_stops))
<br>
<br>
- __ctc_min__ :  Ratio of common_token_count to min lenghth of token count of Q1 and Q2<br>ctc_min = common_token_count / (min(len(q1_tokens), len(q2_tokens))
<br>
<br>

- __ctc_max__ :  Ratio of common_token_count to max lenghth of token count of Q1 and Q2<br>ctc_max = common_token_count / (max(len(q1_tokens), len(q2_tokens))
<br>
<br>
        
- __last_word_eq__ :  Check if First word of both questions is equal or not<br>last_word_eq = int(q1_tokens[-1] == q2_tokens[-1])
<br>
<br>

- __first_word_eq__ :  Check if First word of both questions is equal or not<br>first_word_eq = int(q1_tokens[0] == q2_tokens[0])
<br>
<br>
        
- __abs_len_diff__ :  Abs. length difference<br>abs_len_diff = abs(len(q1_tokens) - len(q2_tokens))
<br>
<br>

- __mean_len__ :  Average Token Length of both Questions<br>mean_len = (len(q1_tokens) + len(q2_tokens))/2
<br>
<br>


- __fuzz_ratio__ 
<br>
<br>

- __fuzz_partial_ratio__ 
<br>
<br>


- __token_sort_ratio__ 
<br>
<br>


- __token_set_ratio__ 
<br>
<br>





- __longest_substr_ratio__ :  Ratio of length longest common substring to min length of token count of Q1 and Q2<br>longest_substr_ratio = len(longest common substring) / (min(len(q1_tokens), len(q2_tokens))


In [ ]:
%%bash
pip install distance

In [ ]:
import distance

In [ ]:
def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features

# get the Longest Common sub string

def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)

def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").apply(preprocess)
    df["question2"] = df["question2"].fillna("").apply(preprocess)

    print("token features...")
    
    # Merging Features with dataset
    
    token_features = df.apply(lambda x: get_token_features(x["question1"], x["question2"]), axis=1)
    
    df["cwc_min"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max"]       = list(map(lambda x: x[1], token_features))
    df["csc_min"]       = list(map(lambda x: x[2], token_features))
    df["csc_max"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    df["mean_len"]      = list(map(lambda x: x[9], token_features))
   
    #Computing Fuzzy Features and Merging with Dataset
    

    print("fuzzy features..")

    df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    # The token sort approach involves tokenizing the string in question, sorting the tokens alphabetically, and 
    # then joining them back into a string We then compare the transformed strings with a simple ratio().
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df

In [ ]:
%%bash
pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
df=extract_features(df)
df.head()

In [ ]:
df.to_csv("nlp_features_train.csv", index=False)

### Visualizing our data after feature engineering

In [ ]:
n = df.shape[0]
sns.pairplot(df[['ctc_min', 'cwc_min', 'csc_min', 'token_sort_ratio', 'is_duplicate']][0:n], hue='is_duplicate', vars=['ctc_min', 'cwc_min', 'csc_min', 'token_sort_ratio'])
plt.show()

In [ ]:
# Distribution of the token_sort_ratio
plt.figure(figsize=(10, 8))

plt.subplot(1,2,1)
plt.title('Violin plot for token_sort_ratio')
sns.violinplot(x = 'is_duplicate', y = 'token_sort_ratio', data = df[0:] , )

plt.subplot(1,2,2)
plt.title('PDF for token_sort_ratio')
sns.distplot(df[df['is_duplicate'] == 1.0]['token_sort_ratio'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['token_sort_ratio'][0:] , label = "0" , color = 'blue' )
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(1,2,1)
plt.title('Violin plot for fuzz_ratio')
sns.violinplot(x = 'is_duplicate', y = 'fuzz_ratio', data = df[0:] , )

plt.subplot(1,2,2)
plt.title('PDF for fuzz ratio')
sns.distplot(df[df['is_duplicate'] == 1.0]['fuzz_ratio'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['fuzz_ratio'][0:] , label = "0" , color = 'blue' )
plt.show()

### We can see most of our newly generated features are useful in the classification task.
### Let's now see , if our data is seperable in n-dimensional space or not

In [ ]:
df=pd.read_csv('nlp_features_train.csv')

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

dfp_subsampled = df[0:10000]
X = MinMaxScaler().fit_transform(dfp_subsampled[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
y = dfp_subsampled['is_duplicate'].values

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne2d = TSNE(
    n_components=2,
    init='random', # pca
    random_state=101,
    method='barnes_hut',
    n_iter=1000,
    verbose=2,
    angle=0.5
).fit_transform(X)

In [ ]:
df2 = pd.DataFrame({'x':tsne2d[:,0], 'y':tsne2d[:,1] ,'label':y})

# draw the plot in appropriate place in the grid
sns.lmplot(data=df2, x='x', y='y', hue='label', fit_reg=False, size=8,palette="Set1",markers=['s','o'])
plt.title("perplexity : {} and max_iter : {}".format(30, 1000))
plt.show()

In [ ]:
df.head()

In [ ]:
import spacy

In [ ]:
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

### To convert questions (text) into numeric value , we'll avg TF-IDF Glove Vector.
### GLOVE is similar to W2V , except It is trained on Wikipedia and therefore, it is stronger in terms of word semantics.

In [ ]:
import tqdm

In [ ]:
df.shape

In [ ]:
y=df['is_duplicate']
df.drop('is_duplicate',axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(df, y, stratify=y, test_size=0.3)

In [ ]:
print("Number of data points in train data :",X_train.shape)
print("Number of data points in test data :",X_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in list(df['question1']):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


In [ ]:
vecs2 = []
for qu2 in list(df['question2']):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv('final_training.csv',index=False)